# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-04-09 19:03:52] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=34951, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=874136191, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=No

[2025-04-09 19:04:01 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-09 19:04:01 TP0] Init torch distributed begin.


[2025-04-09 19:04:03 TP0] Init torch distributed ends. mem usage=0.09 GB
[2025-04-09 19:04:03 TP0] Load weight begin. avail mem=59.40 GB
[2025-04-09 19:04:03 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-09 19:04:05 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.34s/it]


...

[2025-04-09 19:04:08 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-09 19:04:09] INFO:     Started server process [3347052]
[2025-04-09 19:04:09] INFO:     Waiting for application startup.
[2025-04-09 19:04:09] INFO:     Application startup complete.
[2025-04-09 19:04:09] INFO:     Uvicorn running on http://0.0.0.0:34951 (Press CTRL+C to quit)


[2025-04-09 19:04:10] INFO:     127.0.0.1:50040 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-09 19:04:10] INFO:     127.0.0.1:50050 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-09 19:04:10 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:04:13] INFO:     127.0.0.1:50058 - "POST /generate HTTP/1.1" 200 OK
[2025-04-09 19:04:13] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-09 19:04:15 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:04:16 TP0] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 5.21, #queue-req: 0, 


[2025-04-09 19:04:16 TP0] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 97.46, #queue-req: 0, 


[2025-04-09 19:04:17 TP0] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 100.83, #queue-req: 0, 


[2025-04-09 19:04:17 TP0] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 107.54, #queue-req: 0, 


...

[2025-04-09 19:04:34 TP0] Decode batch. #running-req: 1, #token: 1892, token usage: 0.09, gen throughput (token/s): 103.39, #queue-req: 0, 


[2025-04-09 19:04:35 TP0] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 103.97, #queue-req: 0, 


[2025-04-09 19:04:35 TP0] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 106.12, #queue-req: 0, 


[2025-04-09 19:04:35 TP0] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 100.17, #queue-req: 0, 


[2025-04-09 19:04:36 TP0] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 106.08, #queue-req: 0, 
[2025-04-09 19:04:36] INFO:     127.0.0.1:50074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-09 19:04:36 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:04:36 TP0] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 72.85, #queue-req: 0, 


[2025-04-09 19:04:37 TP0] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 103.83, #queue-req: 0, 


[2025-04-09 19:04:37 TP0] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 101.30, #queue-req: 0, 


[2025-04-09 19:04:37 TP0] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 104.50, #queue-req: 0, 


...

[2025-04-09 19:04:58 TP0] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 101.49, #queue-req: 0, 


[2025-04-09 19:04:58 TP0] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 85.62, #queue-req: 0, 


[2025-04-09 19:04:58 TP0] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 86.71, #queue-req: 0, 


[2025-04-09 19:04:59 TP0] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 62.28, #queue-req: 0, 


[2025-04-09 19:04:59] INFO:     127.0.0.1:50074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-09 19:05:00 TP0] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:05:00 TP0] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 56.22, #queue-req: 0, 


[2025-04-09 19:05:00 TP0] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 64.53, #queue-req: 0, 


[2025-04-09 19:05:01 TP0] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 80.13, #queue-req: 0, 


[2025-04-09 19:05:01 TP0] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, gen throughput (token/s): 108.54, #queue-req: 0, 


[2025-04-09 19:05:02] INFO:     127.0.0.1:50074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-09 19:05:02 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-09 19:05:02 TP0] Decode batch. #running-req: 1, #token: 17, token usage: 0.00, gen throughput (token/s): 93.72, #queue-req: 0, 


[2025-04-09 19:05:02 TP0] Decode batch. #running-req: 1, #token: 57, token usage: 0.00, gen throughput (token/s): 106.48, #queue-req: 0, 


[2025-04-09 19:05:02 TP0] Decode batch. #running-req: 1, #token: 97, token usage: 0.00, gen throughput (token/s): 106.40, #queue-req: 0, 


[2025-04-09 19:05:03 TP0] Decode batch. #running-req: 1, #token: 137, token usage: 0.01, gen throughput (token/s): 105.91, #queue-req: 0, 


[2025-04-09 19:05:03] INFO:     127.0.0.1:50074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-09 19:05:03 TP0] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-09 19:05:04 TP0] Decode batch. #running-req: 1, #token: 488, token usage: 0.02, gen throughput (token/s): 54.37, #queue-req: 0, 


[2025-04-09 19:05:04 TP0] Decode batch. #running-req: 1, #token: 528, token usage: 0.03, gen throughput (token/s): 103.89, #queue-req: 0, 


[2025-04-09 19:05:04 TP0] Decode batch. #running-req: 1, #token: 568, token usage: 0.03, gen throughput (token/s): 106.85, #queue-req: 0, 


[2025-04-09 19:05:05 TP0] Decode batch. #running-req: 1, #token: 608, token usage: 0.03, gen throughput (token/s): 102.30, #queue-req: 0, 


[2025-04-09 19:05:05 TP0] Decode batch. #running-req: 1, #token: 648, token usage: 0.03, gen throughput (token/s): 104.19, #queue-req: 0, 


...

[2025-04-09 19:08:16 TP0] Decode batch. #running-req: 1, #token: 20328, token usage: 0.99, gen throughput (token/s): 102.00, #queue-req: 0, 


[2025-04-09 19:08:16 TP0] Decode batch. #running-req: 1, #token: 20368, token usage: 0.99, gen throughput (token/s): 104.49, #queue-req: 0, 


[2025-04-09 19:08:17 TP0] Decode batch. #running-req: 1, #token: 20408, token usage: 1.00, gen throughput (token/s): 104.36, #queue-req: 0, 


[2025-04-09 19:08:17 TP0] Decode batch. #running-req: 1, #token: 20448, token usage: 1.00, gen throughput (token/s): 104.22, #queue-req: 0, 


[2025-04-09 19:08:17] INFO:     127.0.0.1:50074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = json.loads(response.json()["text"].split("</think>")[1])
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-04-09 19:08:18 TP0] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-09 19:08:18 TP0] Decode batch. #running-req: 1, #token: 30, token usage: 0.00, gen throughput (token/s): 43.60, #queue-req: 0, 


[2025-04-09 19:08:18 TP0] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 104.57, #queue-req: 0, 


[2025-04-09 19:08:19 TP0] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, gen throughput (token/s): 106.89, #queue-req: 0, 


[2025-04-09 19:08:19 TP0] Decode batch. #running-req: 1, #token: 150, token usage: 0.01, gen throughput (token/s): 109.12, #queue-req: 0, 


[2025-04-09 19:08:20 TP0] Decode batch. #running-req: 1, #token: 190, token usage: 0.01, gen throughput (token/s): 104.13, #queue-req: 0, 


...

[2025-04-09 19:08:23 TP0] Decode batch. #running-req: 1, #token: 590, token usage: 0.03, gen throughput (token/s): 105.34, #queue-req: 0, 


[2025-04-09 19:08:24 TP0] Decode batch. #running-req: 1, #token: 630, token usage: 0.03, gen throughput (token/s): 108.05, #queue-req: 0, 


[2025-04-09 19:08:24 TP0] Decode batch. #running-req: 1, #token: 670, token usage: 0.03, gen throughput (token/s): 105.66, #queue-req: 0, 


[2025-04-09 19:08:25 TP0] Decode batch. #running-req: 1, #token: 710, token usage: 0.03, gen throughput (token/s): 105.76, #queue-req: 0, 


[2025-04-09 19:08:25] INFO:     127.0.0.1:47598 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not exactly sure where the capital of France is, but I think it\'s Paris. Yeah, I remember hearing that Paris is the capital. Let me think about what details I should include. \n\nFirst, the basic info: country, city, population, and maybe some key landmarks. I know the population is around 2 million, but I\'m not sure of the exact number. I think it\'s approximately 2,165,000. As for landmarks, the Eiffel Tower is a must. The Louvre Museum is another famous spot. The Arc de Triomphe is also iconic. Maybe the Seine River is important too since it\'s a major river in the city.\n\nI should structure this in JSON. So, the main object would have a "country" key pointing to "France," and the "capital" key pointing to "Paris." Under "capital," I can have an "info" array that includes population, landmarks

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-04-09 19:08:25 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:08:25 TP0] Decode batch. #running-req: 1, #token: 24, token usage: 0.00, gen throughput (token/s): 97.45, #queue-req: 0, 


[2025-04-09 19:08:25 TP0] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 109.47, #queue-req: 0, 


[2025-04-09 19:08:26 TP0] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, gen throughput (token/s): 104.58, #queue-req: 0, 


[2025-04-09 19:08:26 TP0] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, gen throughput (token/s): 106.03, #queue-req: 0, 


...

[2025-04-09 19:08:43 TP0] Decode batch. #running-req: 1, #token: 1904, token usage: 0.09, gen throughput (token/s): 101.27, #queue-req: 0, 


[2025-04-09 19:08:44 TP0] Decode batch. #running-req: 1, #token: 1944, token usage: 0.09, gen throughput (token/s): 106.37, #queue-req: 0, 


[2025-04-09 19:08:44 TP0] Decode batch. #running-req: 1, #token: 1984, token usage: 0.10, gen throughput (token/s): 102.03, #queue-req: 0, 


[2025-04-09 19:08:44 TP0] Decode batch. #running-req: 1, #token: 2024, token usage: 0.10, gen throughput (token/s): 105.65, #queue-req: 0, 


[2025-04-09 19:08:45] INFO:     127.0.0.1:47604 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-04-09 19:08:45 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-09 19:08:45 TP0] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:08:45 TP0] Decode batch. #running-req: 3, #token: 41, token usage: 0.00, gen throughput (token/s): 99.75, #queue-req: 0, 


[2025-04-09 19:08:45 TP0] Decode batch. #running-req: 3, #token: 161, token usage: 0.01, gen throughput (token/s): 287.69, #queue-req: 0, 


[2025-04-09 19:08:46 TP0] Decode batch. #running-req: 3, #token: 281, token usage: 0.01, gen throughput (token/s): 295.46, #queue-req: 0, 


[2025-04-09 19:08:46 TP0] Decode batch. #running-req: 3, #token: 401, token usage: 0.02, gen throughput (token/s): 301.29, #queue-req: 0, 


...

[2025-04-09 19:09:04 TP0] Decode batch. #running-req: 3, #token: 5681, token usage: 0.28, gen throughput (token/s): 293.53, #queue-req: 0, 


[2025-04-09 19:09:05 TP0] Decode batch. #running-req: 3, #token: 5801, token usage: 0.28, gen throughput (token/s): 288.11, #queue-req: 0, 


[2025-04-09 19:09:05 TP0] Decode batch. #running-req: 3, #token: 5921, token usage: 0.29, gen throughput (token/s): 299.50, #queue-req: 0, 


[2025-04-09 19:09:05 TP0] Decode batch. #running-req: 3, #token: 6041, token usage: 0.29, gen throughput (token/s): 294.12, #queue-req: 0, 


[2025-04-09 19:09:06] INFO:     127.0.0.1:50914 - "POST /generate HTTP/1.1" 200 OK
[{'text': "600 words.\n\nThe capital of France is Paris. Paris is one of the most important cities in the world, and it's also the political, cultural, and economic center of France. The city has a rich history that dates back to ancient times, and it's known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. Paris is also famous for its cuisine, with dishes like baguette, croissant, and boeuf bourguignon being some of the most popular. The city is surrounded by the Seine River, which flows through it, and the bridges over the river add to the city's charm. Paris is a vibrant city with a mix of old-world charm and modern innovation, making it a unique and fascinating place to visit.\n\nThe capital of France is Paris. Paris is one of the most important cities in the world, and it's also the political, cultural, and economic center of France. The city has a rich 

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-04-09 19:09:06 TP0] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-09 19:09:06 TP0] Decode batch. #running-req: 1, #token: 8, token usage: 0.00, gen throughput (token/s): 270.20, #queue-req: 0, 


[2025-04-09 19:09:06 TP0] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 104.60, #queue-req: 0, 


[2025-04-09 19:09:07 TP0] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 104.18, #queue-req: 0, 


[2025-04-09 19:09:07 TP0] Decode batch. #running-req: 1, #token: 128, token usage: 0.01, gen throughput (token/s): 103.56, #queue-req: 0, 


[2025-04-09 19:09:07 TP0] Decode batch. #running-req: 1, #token: 168, token usage: 0.01, gen throughput (token/s): 103.69, #queue-req: 0, 


...

[2025-04-09 19:09:24 TP0] Decode batch. #running-req: 1, #token: 1888, token usage: 0.09, gen throughput (token/s): 102.90, #queue-req: 0, 


[2025-04-09 19:09:24 TP0] Decode batch. #running-req: 1, #token: 1928, token usage: 0.09, gen throughput (token/s): 103.19, #queue-req: 0, 


[2025-04-09 19:09:25 TP0] Decode batch. #running-req: 1, #token: 1968, token usage: 0.10, gen throughput (token/s): 105.91, #queue-req: 0, 


[2025-04-09 19:09:25 TP0] Decode batch. #running-req: 1, #token: 2008, token usage: 0.10, gen throughput (token/s): 104.92, #queue-req: 0, 


[2025-04-09 19:09:25 TP0] Decode batch. #running-req: 1, #token: 2048, token usage: 0.10, gen throughput (token/s): 108.16, #queue-req: 0, 
[2025-04-09 19:09:25] INFO:     127.0.0.1:38850 - "POST /generate HTTP/1.1" 200 OK
{'text': ' the \\( n \\9121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-04-09 19:09:25 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 19:09:26 TP0] Decode batch. #running-req: 1, #token: 54, token usage: 0.00, gen throughput (token/s): 99.28, #queue-req: 0, 


[2025-04-09 19:09:26 TP0] Decode batch. #running-req: 1, #token: 94, token usage: 0.00, gen throughput (token/s): 105.13, #queue-req: 0, 


[2025-04-09 19:09:26 TP0] Decode batch. #running-req: 1, #token: 134, token usage: 0.01, gen throughput (token/s): 105.06, #queue-req: 0, 


[2025-04-09 19:09:27 TP0] Decode batch. #running-req: 1, #token: 174, token usage: 0.01, gen throughput (token/s): 103.93, #queue-req: 0, 


...

[2025-04-09 19:09:31 TP0] Decode batch. #running-req: 1, #token: 574, token usage: 0.03, gen throughput (token/s): 102.76, #queue-req: 0, 


[2025-04-09 19:09:31 TP0] Decode batch. #running-req: 1, #token: 614, token usage: 0.03, gen throughput (token/s): 100.80, #queue-req: 0, 


[2025-04-09 19:09:32 TP0] Decode batch. #running-req: 1, #token: 654, token usage: 0.03, gen throughput (token/s): 105.30, #queue-req: 0, 


[2025-04-09 19:09:32 TP0] Decode batch. #running-req: 1, #token: 694, token usage: 0.03, gen throughput (token/s): 103.09, #queue-req: 0, 


[2025-04-09 19:09:32 TP0] Decode batch. #running-req: 1, #token: 734, token usage: 0.04, gen throughput (token/s): 100.72, #queue-req: 0, 
[2025-04-09 19:09:32] INFO:     127.0.0.1:59052 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-04-09 19:09:32] Child process unexpectedly failed with an exit code 9. pid=3347789


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.23s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.13s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.14s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text:  and also, make sure that the JSON is valid.

```json
{
  "name": "Beijing",
  "population": 10000000,
  "area": 100000,
  "founded": 1500,
  "coordinates": {
    "latitude": "40.4168",
    "longitude": "-73.9352"
  }
}
```

Is this JSON valid? If not, explain why.

If it is valid, explain why.

Also, provide an updated version of the JSON with any necessary corrections.
</think>{

"name": "Beijing",
"population": 1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text:  and also, make sure that the JSON is valid.

```json
{
  "name": "Beijing",
  "population": 10000000,
  "area": 100000,
  "founded": 1500,
  "coordinates": {
    "latitude": "40.4168",
    "longitude": "-73.9352"
  }
}
```

Is this JSON valid? If not, explain why.

If it is valid, explain why.

Also, provide an updated version of the JSON with any necessary corrections.
</think>{

"name": "Beijing",
"population": 1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 900 characters only.
Okay, I need to find out the information about the capital of France, which is Paris, but the user specified exactly 900 characters. Hmm, let me think.

First, Paris is the capital, so that's straightforward. It's located in Île-de-France, which is a major island in the River Seine. The exact coordinates are important—latitude and longitude. Paris has been the capital since the early 19th century, right? Before that, it was another city.

I remember that Paris was called Paris until the early 19th century, so maybe the
Prompt: Give me the information of the capital of Germany.
Generated text: 600 characters.

The user is asking for information about the capital of Germany, which is Berlin. They specified a character limit of 600, but the provided response is only 200 words. However, they might be expecting more detailed information. They're probably looking for a concise yet comprehensive ove

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its population, economy, culture, and major landmarks.

}
Okay, so I need to figure out information about London as a major global city. The user has specified population, economy, culture, and major landmarks. Let me start by recalling what I know about London.

First, I know London is the capital of the United Kingdom and has been a major city for a long time. I think the population is over 8 million, but I'm not entirely sure if it's exactly 9 million. I remember seeing it mentioned that London is the most populous city in the European Union. That makes sense because the EU covers a lot of
Prompt: Please provide information about Paris as a major global city:
Generated text:  9 sentences, each about a different aspect, and then conclude with a 5-sentence paragraph summarizing the essence of Paris as a global hub.

The first sentence must be about the name of the city, including its origin.

Seco

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Okay, I need to provide the JSON format for the information of Paris, the capital of France. Let me think about what details are essential. First, the country is France, so that's straightforward. The capital is Paris, so that's the obvious part.

Next, the population. As of the latest estimates, Paris has around 2.1 million people. I should make sure to check the most recent data, but I believe that's accurate. 

Language is another key point. Paris primarily uses French, so I'll include that. The currency is the Euro, commonly used there, though maybe not as widely as other major cities, but it's standard for the country.

The official website is the État de France, which is the government's portal, but sometimes they mention the Ile-de-France region's website as the city's official site. I should note that as well.

The mayor's name is Ali C

In [19]:
llm.shutdown()